In [1]:
import pandas as pd
import numpy as np
import os


import pandas_datareader as pdr  # to install: !pip install pandas_datareader
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt 
from scipy import stats
from sklearn.linear_model import LinearRegression

from statsmodels.formula.api import ols as sm_ols
from statsmodels.iolib.summary2 import summary_col # nicer tables
import statsmodels.api as sm

# importing required modules
from zipfile import ZipFile
import zipfile
import time

What i'd use if the file loaded in faster 

In [2]:
# file_name = 'stataversion.dta'
# returns_data = pd.read_stata(file_name) #reads in the data file 
# returns_data.head()

In [3]:
# file_name2 = 'hpbx8j61yn2jkyar.dta'
# fundamental_data = pd.read_stata(file_name2) #reads in the data file 
# fundamental_data.head()

Covert to csv then read in that csv... apperently works faster

In [4]:

# reader=pd.read_stata("hpbx8j61yn2jkyar.dta",chunksize=100000) #read dta file in chunks

# df = pd.DataFrame()

# for itm in reader:
#     df=df.append(itm)

# df.to_csv("fundamentals.csv") #convert it to a csv 


In [5]:
# reader=pd.read_stata("stataversion.dta",chunksize=100000)

# df = pd.DataFrame()

# for itm in reader:
#     df=df.append(itm)

# df.to_csv("returns.csv")

In [6]:
#Compustat data - roughly 1min to load 
fundamentals = pd.concat((chunk for chunk in pd.read_csv('fundamentals.csv',chunksize=100000, low_memory=False)))
fundamentals.shape #~500k rows, 659 columns 

(501147, 659)

In [7]:
fundamentals['cusip']
#CUSIP 9 digits: first 6 match the company, next 2 match stock offering, last 1 checks digit 

0         000361105
1         000361105
2         000361105
3         000361105
4         000361105
            ...    
501142    21077C107
501143    21077C107
501144    21077C107
501145    21077C107
501146    21077C107
Name: cusip, Length: 501147, dtype: object

In [8]:
#CRSP data - roughly 10 seconds to load 
returns = pd.concat((chunk for chunk in pd.read_csv('returns.csv',chunksize=100000, low_memory=False)))
returns.shape #~5.7 million rows, 16 columns

(5711711, 16)

In [57]:
#cleaning up returns dataset - roughly 10 sec to load

returns['date'] = pd.to_datetime(returns['date'], format='%Y%m%d', errors='coerce') #change dates format 
returns.set_index('date', inplace=True)
returns.sort_values(by='date') # This now sorts in date order
returns #returns in number format, not percentages 
returns[['prc','vol','ret','shrout','sprtrn']] = returns[['prc','vol','ret','shrout','sprtrn']].apply(pd.to_numeric, errors='coerce') #had to change the num to to numeric values 
#returns['ticker'].nunique() #have stock return data on 9,670 firms 
returns.head()

KeyError: 'date'

In [53]:
#returns.resample('Q').mean()
#returns

In [11]:
fundamentals.sort_values(by='fyearq').reset_index() # This now sorts in date order
#fundamentals.insert(0, 'cusip', fundamentals.cusip(1))

fundamentals.tail(20)

,Unnamed: 0,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,...,prirow,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,ipodate
501127,501127,345920,2019-03-31,2019.0,1.0,12.0,INDL,C,D,STD,...,NaN,1.0,3524.0,NaN,NaN,NaN,PA,0.0,www.hydrofarm.com,2020-12-10
501128,501128,345920,2019-06-30,2019.0,2.0,12.0,INDL,C,D,STD,...,NaN,1.0,3524.0,NaN,NaN,NaN,PA,0.0,www.hydrofarm.com,2020-12-10
501129,501129,345920,2019-09-30,2019.0,3.0,12.0,INDL,C,D,STD,...,NaN,1.0,3524.0,NaN,NaN,NaN,PA,0.0,www.hydrofarm.com,2020-12-10
501130,501130,345920,2019-12-31,2019.0,4.0,12.0,INDL,C,D,STD,...,NaN,1.0,3524.0,NaN,NaN,NaN,PA,0.0,www.hydrofarm.com,2020-12-10
501131,501131,345920,2020-03-31,2020.0,1.0,12.0,INDL,C,D,STD,...,NaN,1.0,3524.0,NaN,NaN,NaN,PA,0.0,www.hydrofarm.com,2020-12-10
501132,501132,345920,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,NaN,1.0,3524.0,NaN,NaN,NaN,PA,0.0,www.hydrofarm.com,2020-12-10
501133,501133,345920,2020-09-30,2020.0,3.0,12.0,INDL,C,D,STD,...,NaN,1.0,3524.0,NaN,NaN,NaN,PA,0.0,www.hydrofarm.com,2020-12-10
501134,501134,345920,2020-12-31,2020.0,4.0,12.0,INDL,C,D,STD,...,NaN,1.0,3524.0,NaN,NaN,NaN,PA,0.0,www.hydrofarm.com,2020-12-10
501135,501135,345980,2018-03-31,2018.0,1.0,12.0,INDL,C,D,STD,...,NaN,1.0,5961.0,NaN,NaN,NaN,CA,0.0,www.wish.com,2020-12-16
501136,501136,345980,2018-06-30,2018.0,2.0,12.0,INDL,C,D,STD,...,NaN,1.0,5961.0,NaN,NaN,NaN,CA,0.0,www.wish.com,2020-12-16


In [12]:
returns.columns
#permno: unique stock (share class) identifier
#siccd: industry classification code
#ncusip: CUSIP identifier for stock
#comnam: company name 
#shrcls: share class
#naics: north american industry classification code
#prc: price
#vol: volume
#ret: daily return 
#sprtrn: sp500 daily return 

Index(['Unnamed: 0', 'permno', 'date', 'siccd', 'ncusip', 'ticker', 'comnam',
       'shrcls', 'naics', 'prc', 'vol', 'ret', 'shrout', 'cfacpr', 'cfacshr',
       'sprtrn'],
      dtype='object')

In [13]:
#null values for each variable 
print(returns.isnull().sum()) 


Unnamed: 0          0
permno              0
date                0
siccd              88
ncusip             88
ticker          12168
comnam             88
shrcls        5283496
naics          101883
prc             15009
vol             15008
ret             17136
shrout           1281
cfacpr           1281
cfacshr          1281
sprtrn              0
dtype: int64


In [14]:
#group returns by quarter
#group return dates by 3/31, 6/30, 9/30, 12/31 


returns['Gross_Ret'] = 1 + returns['ret']

q2_2018 = ((returns[(returns['date'] >= '2018-03-31') & (returns['date'] <= '2018-06-30')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q3_2018 = ((returns[(returns['date'] >= '2018-06-30') & (returns['date'] <= '2018-09-30')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q4_2018 = ((returns[(returns['date'] >= '2018-09-30') & (returns['date'] <= '2018-12-31')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q1_2019 = ((returns[(returns['date'] >= '2018-12-31') & (returns['date'] <= '2019-03-31')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 

q2_2019 = ((returns[(returns['date'] >= '2019-03-31') & (returns['date'] <= '2019-06-30')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q3_2019 = ((returns[(returns['date'] >= '2019-06-30') & (returns['date'] <= '2019-09-30')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q4_2019 = ((returns[(returns['date'] >= '2019-09-30') & (returns['date'] <= '2019-12-31')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q1_2020 = ((returns[(returns['date'] >= '2019-12-31') & (returns['date'] <= '2020-03-31')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 

q2_2020 = ((returns[(returns['date'] >= '2020-03-31') & (returns['date'] <= '2020-06-30')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q3_2020 = ((returns[(returns['date'] >= '2020-06-30') & (returns['date'] <= '2020-09-30')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q4_2020 = ((returns[(returns['date'] >= '2020-09-30') & (returns['date'] <= '2020-12-31')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 

In [54]:
q4_2020

,ncusip,ticker,Gross_Ret
0,00036020,AAON,0.121366
1,00036110,AIR,0.895349
2,00037520,ABB,0.091759
3,00038020,ABCM,0.091138
4,00081T10,ACCO,0.429465
...,...,...,...
8197,Y8564M10,TGP,0.130567
8198,Y8564W10,TK,-0.018263
8199,Y8565N30,TNK,0.015685
8200,Y8897Y18,TOPS,0.120372


In [15]:
#need to figure out how to add the quarterly returns for each firm for each resepective previous quarter 

#good practice but incorrect merge: every observation from returns to every observation in fundamentals 

#writing out steps i think i need to take...
- remove the 2020q4 data since the last time period we want to measure is 2020q4 stock returns on 2020q3 financial data 
- need to merge the quarterly returns for each firm with each respective datadate acct data released 

- if datadate = 2018-03-31, merge q2_2018 where each return is matched to each ticker 

In [16]:
fundamentals['cusip'].describe() #count 500,731... 19,122 unique
returns['ncusip'].describe() #count 5,711,623... 10,650 unique 

#cusip with ncusp

count      5711623
unique       10650
top       09073Q10
freq           756
Name: ncusip, dtype: object

In [17]:
fundamentals['cusip'].head(15)
fundamentals[['cusip','tic']].nunique() #19,122 cusip matches, 12,120 tics


cusip    19122
tic      19120
dtype: int64

In [18]:
returns['ncusip'].head(15)
returns[['ncusip','ticker']].nunique() #10,650 ncusip matches, 9,670 tickers  


ncusip    10650
ticker     9670
dtype: int64

In [19]:
q2_2020

,ncusip,ticker,Gross_Ret
0,00036020,AAON,0.128887
1,00036110,AIR,0.142619
2,00037520,ABB,0.300290
3,00081T10,ACCO,0.435157
4,00086810,ACNB,-0.035611
...,...,...,...
7828,Y8564M10,TGP,0.246022
7829,Y8564W10,TK,-0.225806
7830,Y8565N30,TNK,-0.470903
7831,Y8897Y16,TOPS,-0.745508


In [20]:
fundamentals.tail()

,Unnamed: 0,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,...,prirow,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,ipodate
501142,501142,345980,2019-12-31,2019.0,4.0,12.0,INDL,C,D,STD,...,NaN,1.0,5961.0,NaN,NaN,NaN,CA,0.0,www.wish.com,2020-12-16
501143,501143,345980,2020-03-31,2020.0,1.0,12.0,INDL,C,D,STD,...,NaN,1.0,5961.0,NaN,NaN,NaN,CA,0.0,www.wish.com,2020-12-16
501144,501144,345980,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,NaN,1.0,5961.0,NaN,NaN,NaN,CA,0.0,www.wish.com,2020-12-16
501145,501145,345980,2020-09-30,2020.0,3.0,12.0,INDL,C,D,STD,...,NaN,1.0,5961.0,NaN,NaN,NaN,CA,0.0,www.wish.com,2020-12-16
501146,501146,345980,2020-12-31,2020.0,4.0,12.0,INDL,C,D,STD,...,NaN,1.0,5961.0,NaN,NaN,NaN,CA,0.0,www.wish.com,2020-12-16


In [21]:
#     if fundamentals[fundamentals["datadate"] == "2020-06-30"]:
#         Merge = pd.merge(fundamentals, q3_2020a,left_on='cusip', right_on='ncusip',indicator=True, validate='one_to_one')
   
    
# Merge


# # >>> df[(df['py-score'] >= 80) & (df['js-score'] >= 80)]


In [22]:
# Merge = pd.merge(fundamentals.loc[fundamentals['datadate']]=='2020-06-30', q3_2020a,left_on='cusip', right_on='ncusip', how='right',indicator=True,validate='many_to_one')
# Merge.tail(20)

q1_2020_acct = fundamentals[fundamentals['datadate'] == '2020-03-31']
q2_2020_acct = fundamentals[fundamentals['datadate'] == '2020-06-30']
q3_2020_acct = fundamentals[fundamentals['datadate'] == '2020-09-30']
q4_2020_acct = fundamentals[fundamentals['datadate'] == '2020-12-31']
q1_2020_acct = fundamentals[fundamentals['datadate'] == '2020-03-31']
q2_2020_acct = fundamentals[fundamentals['datadate'] == '2020-06-30']
q3_2020_acct = fundamentals[fundamentals['datadate'] == '2020-09-30']
q4_2020_acct = fundamentals[fundamentals['datadate'] == '2020-12-31']

q2_2020_acct


#df[(df['date'] > '2013-01-01') & (df['date'] < '2013-02-01')]

# Merge = pd.merge(q2_2020_acct, q3_2020,left_on='cusip', right_on='ncusip',how='left',indicator=True)
# Merge

,Unnamed: 0,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,...,prirow,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,ipodate
89,89,1045,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,NaN,4.0,4512.0,605.0,600.0,C,TX,0.0,www.aa.com,NaN
133,133,1050,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,NaN,1.0,3564.0,345.0,925.0,C,TX,0.0,www.cecoenviro.com,NaN
260,260,1075,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,NaN,1.0,4911.0,705.0,700.0,A,AZ,0.0,www.pinnaclewest.com,NaN
304,304,1076,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,NaN,1.0,6141.0,175.0,976.0,B,UT,0.0,NaN,NaN
348,348,1078,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,NaN,1.0,3845.0,280.0,905.0,B,IL,0.0,www.abbott.com,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501082,501082,330942,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,NaN,1.0,6722.0,NaN,NaN,NaN,NY,NaN,am.jpmorgan.com,NaN
501091,501091,331856,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,NaN,1.0,2836.0,NaN,NaN,NaN,NY,0.0,www.immunic-therapeutics.com,NaN
501099,501099,332115,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,NaN,1.0,2836.0,NaN,NaN,NaN,CA,0.0,www.armatapharma.com,NaN
501132,501132,345920,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,NaN,1.0,3524.0,NaN,NaN,NaN,PA,0.0,www.hydrofarm.com,2020-12-10


In [26]:
#returns['ncusip'].isin(fundamentals['cusip']).value_counts() #ncusip appears in fundamental 88 times 
#fundamentals['cusip'].isin(returns['ncusip']).value_counts() #cusip appears in returns 416 matches 

#q3_2020['ncusip'].isin(q2_2020_acct['cusip']).value_counts() #0 times!!!! 0 for acct q1,q3,q4 as well 
#q3_2020_acct['cusip'].isin(q3_2020['ncusip']).value_counts() #0 times!!!! 0 for acct q1,q2,q3,q4 as well 

In [23]:
fundamentals['cusip'].isin(returns['ncusip']).value_counts() #cusip appears in returns 416 matches 
returns['ncusip'].isin(fundamentals['cusip']).value_counts() #ncusip appears in fundamental 88 times 

False    5711623
True          88
Name: ncusip, dtype: int64

In [11]:
fundamentals[fundamentals['cusip'].isin(returns['ncusip'])]['cusip']

78197     NaN
78198     NaN
78199     NaN
78200     NaN
78201     NaN
         ... 
434187    NaN
436362    NaN
436363    NaN
436364    NaN
453098    NaN
Name: cusip, Length: 416, dtype: object

In [34]:
fundamentals_a = fundamentals
fundamentals_a['cusip'] = fundamentals_a['cusip'].astype(str).str[:-1]

In [35]:
q2_2020_acct_a = fundamentals_a[fundamentals_a['datadate'] == '2020-06-30']
q2_2020_acct_a['cusip']
q3_2020

89        02376R10
133       12514110
260       72348410
304       74319R10
348       00282410
            ...   
501082    46641Q39
501091    4525EP10
501099    04216R10
501132    44888K20
501144    21077C10
Name: cusip, Length: 9776, dtype: object

In [36]:
Merge = pd.merge(q2_2020_acct_a, q3_2020,left_on='cusip', right_on='ncusip',how='left',indicator=True)
Merge

,Unnamed: 0,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,...,spcseccd,spcsrc,state,stko,weburl,ipodate,ncusip,ticker,Gross_Ret,_merge
0,89,1045,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,600.0,C,TX,0.0,www.aa.com,NaN,02376R10,AAL,-0.077326,both
1,133,1050,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,925.0,C,TX,0.0,www.cecoenviro.com,NaN,12514110,CECE,0.107902,both
2,260,1075,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,700.0,A,AZ,0.0,www.pinnaclewest.com,NaN,72348410,PNW,0.031416,both
3,304,1076,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,976.0,B,UT,0.0,NaN,NaN,NaN,NaN,NaN,left_only
4,348,1078,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,905.0,B,IL,0.0,www.abbott.com,NaN,00282410,ABT,0.227310,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9779,501082,330942,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,NaN,NaN,NY,NaN,am.jpmorgan.com,NaN,46641Q39,BBUS,0.110585,both
9780,501091,331856,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,NaN,NaN,NY,0.0,www.immunic-therapeutics.com,NaN,4525EP10,IMUX,0.486790,both
9781,501099,332115,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,NaN,NaN,CA,0.0,www.armatapharma.com,NaN,04216R10,ARMP,-0.212344,both
9782,501132,345920,2020-06-30,2020.0,2.0,12.0,INDL,C,D,STD,...,NaN,NaN,PA,0.0,www.hydrofarm.com,2020-12-10,NaN,NaN,NaN,left_only
